# Ініціалізація проекту

In [ ]:
from pprint import pprint # не забыть удалить

In [1]:
!pip install SpeechRecognition

     |████████████████████████████████| 32.8 MB 38 kB/s 


In [ ]:
!pip install urllib
!pip install requests
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install SpeechRecognition

ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib
     |████████████████████████████████| 50 kB 3.0 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149861 sha256=b20f86c811ca66b3779b5d4ce2c7937ec9b030b858a1626c25e598271f31f164
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/Study/DevNet/HikkaShop']

In [ ]:
sys.path.append("/content/drive/MyDrive/Study/DevNet/HikkaShop")

# Модуль розпізнавання мовлення (Speech Recognition)

In [5]:
import speech_recognition as sr
speech_folder = "speechInput/"
speech_file = "audio_files_harvard.wav"
audioInput = sr.AudioFile(speech_folder+speech_file)
r = sr.Recognizer()

with audioInput as source:
  audio = r.record(source)
  text = r.recognize_google(audio)
  print(text)

the stale smell of old beer lingers it takes heat to bring out the odour a cold dip restores health exist a salt pickle taste fine with him pastura my favourite exist for food is the hot cross bun


# Модуль обробки природної мови (NLP)

## Функція отримання відповіді від WaifuAI Api

In [ ]:
import requests as re
import urllib.parse
import Keys

def get_response(from_name, to_name, situation: list, user_id, message):
  data_dict = {
    "from_name": from_name,
    "to_name": to_name,
    "situation": ' '.join(map(str, situation)),
    "message": message,

    "user_id": user_id,
    "preset_mode": "waifu",
    "translate_from": "uk",
  }

  params = {
    "url": "https://waifu-ai.p.rapidapi.com/path",
    "headers": {
      'content-type': 'application/x-www-form-urlencoded',
      'x-rapidapi-host': 'waifu-ai.p.rapidapi.com',
      'x-rapidapi-key': Keys.keyMax
    },
    "data": "&".join(param+"="+urllib.parse.quote(value) for param, value in data_dict.items())
  }
  
  response = re.post(**params)
  return response.text, response

## Створення логфайлу бесіди

In [ ]:
def logOutput(log):
  from datetime import datetime
  now = datetime.now().strftime("%Y-%m-%d_%H.%M.%S")
  with open("logfiles/"+now+".txt", "a", encoding="UTF-8") as ouf:
    ouf.write(log)

## Суперклас WaifuAI

In [ ]:
class WaifuAI():
  @staticmethod
  def setattrs(base_situation: str, user_id: str = None):
    WaifuAI.base_situation = base_situation
    WaifuAI.situation = [base_situation]

    user_id = None if user_id == "" else user_id
    WaifuAI.user_id = user_id or WaifuAI.generate_user_id()
  
  @staticmethod
  def append_situation(situation: str):
    WaifuAI.situation.append(situation)

  @staticmethod
  def reset_situation():
    WaifuAI.situation = [WaifuAI.base_situation]
  
  def generate_user_id():
    if WaifuAI.user_id is None:
      import string
      import random
      alnum = string.ascii_letters + string.digits
      user_id = ''.join(random.choice(alnum) for i in range(64))
      print(user_id)
      return user_id

## Клас продавця

In [ ]:
class Seller(WaifuAI):
  def __init__(self, name: str, stock: dict):
    self.name = name
    self.stock = stock
    self.sync_stock()

  def sync_stock(self):
    for merch, v in self.stock.items():
      qnty = v[0]
      price = v[1]
      if qnty != 0:
        super().append_situation(f"{self.name} може продати {merch} у кількості {qnty} за ціною {price} гривень.")
      else:
        super().append_situation(f"{merch} закінчився, тому {self.name} не може продати {merch}.")
        
  def get_order(self, message, tolerance=70):
    import re
    from fuzzywuzzy import fuzz

    match_obj = re.match("купити ([\w\s]*)\.", message)
    if match_obj:
      finded_merch = match_obj.groups()[0]

      max_ratio = 0
      merch = None
      for stock_merch in self.stock.keys():
        ratio = fuzz.ratio(finded_merch, stock_merch)

        if ratio > max_ratio or max_ratio == 0:
          max_ratio = ratio
          merch = stock_merch

      if max_ratio >= tolerance and merch is not None:
        return merch
      return "Not Found"
    return None

## Клас покупця

In [ ]:
class Customer(WaifuAI):
  def __init__(self, name: str, cash: float):
    self.name = name
    self.cash = cash
    self.sync_cash()

  def sync_cash(self):
    super().append_situation(f"{self.name} має при собі {self.cash} гривень.")

  def purchase(self, seller: Seller, merch):
    if merch == "Not Found":
      super().situation.append(f"{self.name} намагався купити товар, але {seller.name} не зрозуміла що це за товар.")
      return

    merch_qnty = seller.stock[merch][0]
    merch_price = seller.stock[merch][1]
    
    if merch_qnty == 0:
      super().append_situation(f"{self.name} намагався купити {merch}, але {merch} закінчився.")
      return

    if self.cash < merch_price:
      super().append_situation(f"{self.name} намагався купити {merch}, але {self.name} не має стільки грошей.")
      return

    seller.stock[merch][0] -= 1
    self.cash -= merch_price
    super().reset_situation()
    self.sync_cash()
    seller.sync_stock()
    print("---")
    print(f"Каса-сама: Баланс {self.name} становить {self.cash}.")
    print("---")

## Вхідні дані розмови

In [ ]:
from merch_list import merch_list

#@title Ствоерення діючих осіб{ run: "auto", form-width: "50%" }
#@markdown Покупець
customer_name = "Тарас-кун" #@param {type: "raw"}
cash          = 12.3#@param {type:"number"}

#@markdown Продавець та система
seller_name = "Акено-тян" #@param {type: "raw"}
situation   = "Акено-тян продає аніме-товари в магазині." #@param {type: "raw"}
user_id     = "" #@param {type: "raw"}

## Надсилання повідомлень

In [ ]:
WaifuAI.setattrs(situation, user_id)

customer = Customer(customer_name, cash)
seller = Seller(seller_name, merch_list)

log = f"""from_name = {customer.name}
to_name   = {seller.name}
situation = {WaifuAI.situation}
user_id   = {WaifuAI.user_id}
---\n
"""

print("Каса-сама: Для замовлення товару наберіть 'купити НАЗВА_ТОВАРУ.'")
print("Каса-сама: Для завершення бесіди наберіть 'Пока!'")
print("---")

while(True):
  message = input(f"{customer.name}: ")
  log += f"{customer.name}: {message}\n"

  merch = seller.get_order(message)
  if merch is not None:
    customer.purchase(seller, merch)

  response_text, response = get_response(
    from_name = customer.name,
    to_name   = seller.name,
    situation = WaifuAI.situation,
    user_id   = WaifuAI.user_id,
    message   = message
  )

  sellerResp = f'{seller.name}: {response_text}'
  print(sellerResp)
  log += sellerResp+"\n"

  if (message == "Пока!"):
    logOutput(log)
    break

Каса-сама: Для замовлення товару наберіть 'купити НАЗВА_ТОВАРУ.'
Каса-сама: Для завершення бесіди наберіть 'Пока!'
---
['Акено-тян продає аніме-товари в магазині.',
 'Тарас-кун має при собі 12.3 гривень.',
 'Акено-тян може продати Фігурка Ріас Греморі у кількості 1 за ціною 160 '
 'гривень.',
 'Акено-тян може продати Дакімакура у кількості 5 за ціною 40 гривень.',
 'Акено-тян може продати Манга Британський Кровосіся у кількості 10 за ціною '
 '70 гривень.']
from_name=%D0%A2%D0%B0%D1%80%D0%B0%D1%81-%D0%BA%D1%83%D0%BD&to_name=%D0%90%D0%BA%D0%B5%D0%BD%D0%BE-%D1%82%D1%8F%D0%BD&situation=%D0%90%D0%BA%D0%B5%D0%BD%D0%BE-%D1%82%D1%8F%D0%BD%20%D0%BF%D1%80%D0%BE%D0%B4%D0%B0%D1%94%20%D0%B0%D0%BD%D1%96%D0%BC%D0%B5-%D1%82%D0%BE%D0%B2%D0%B0%D1%80%D0%B8%20%D0%B2%20%D0%BC%D0%B0%D0%B3%D0%B0%D0%B7%D0%B8%D0%BD%D1%96.%20%D0%A2%D0%B0%D1%80%D0%B0%D1%81-%D0%BA%D1%83%D0%BD%20%D0%BC%D0%B0%D1%94%20%D0%BF%D1%80%D0%B8%20%D1%81%D0%BE%D0%B1%D1%96%2012.3%20%D0%B3%D1%80%D0%B8%D0%B2%D0%B5%D0%BD%D1%8C.%20%D0%90%D0%BA%D